In [1]:
#!pip install texthero
import pandas as pd
import numpy as np
import texthero as hero
from texthero import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, sigmoid_kernel
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
def readin(file):
    return pd.read_csv(f"Data/{file}.csv", index_col=0)

In [3]:
df_b_r = readin('df_books_rev')

In [4]:
df_b_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94029 entries, 0 to 94028
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   asin        94029 non-null  object
 1   reviewText  94029 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [5]:
df_b_r.head()

,asin,reviewText
0,B0012GTZCK,Not what I was expecting. I was looking for a ...
1,B0012LHGJ4,IT WAS DONE AS A FILM IN THE 1950-60s. VERY P...
2,B0012RMVH0,Needs pictures Excellent book. This is somethi...
3,B0012TAD1O,She always delivers with an exciting storyline...
4,B0012U0NKE,"<a data-hook=""product-link-linked"" class=""a-li..."


In [6]:
def clean_review(df):
    custom_pipeline = [preprocessing.lowercase,
                  preprocessing.remove_punctuation,
                  preprocessing.remove_stopwords,
                  preprocessing.stem,
                  preprocessing.remove_whitespace]
    df['clean_text'] = hero.clean(df['reviewText'], custom_pipeline)

In [7]:
clean_review(df_b_r)

In [8]:
df_b_r.set_index('asin', inplace=True)

In [9]:
df_b_r.head()

,reviewText,clean_text
asin,,
B0012GTZCK,Not what I was expecting. I was looking for a ...,expect look histori christian influenc live pe...
B0012LHGJ4,IT WAS DONE AS A FILM IN THE 1950-60s. VERY P...,done film 1950 60s popular film back today kid...
B0012RMVH0,Needs pictures Excellent book. This is somethi...,need pictur excel book someth want book give k...
B0012TAD1O,She always delivers with an exciting storyline...,alway deliv excit storylin lot heat cours imag...
B0012U0NKE,"<a data-hook=""product-link-linked"" class=""a-li...",data hook product link link class link normal ...


In [10]:
tf = TfidfVectorizer(max_features=300)
dtm = tf.fit_transform(df_b_r['clean_text'])
dtm = pd.DataFrame(dtm.todense(), columns=tf.get_feature_names(), index=df_b_r.index)

In [11]:
dtm.head()

,abl,absolut,action,actual,adventur,almost,along,alreadi,also,although,...,world,worth,would,write,writer,written,wrong,year,yet,young
asin,,,,,,,,,,,,,,,,,,,,,
B0012GTZCK,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
B0012LHGJ4,0.0,0.0,0.000000,0.000000,0.362908,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.183927,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.327082
B0012RMVH0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.100931,0.0,0.000000
B0012TAD1O,0.0,0.0,0.090261,0.090717,0.000000,0.0,0.0,0.0,0.0,0.098413,...,0.085004,0.0,0.110181,0.066369,0.198286,0.065399,0.0,0.000000,0.0,0.000000
B0012U0NKE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


In [12]:
df_meta_all = pd.read_csv('Data/meta_all.csv', index_col='asin')
df_meta_all.drop(columns =['Unnamed: 0'], inplace=True)
df_meta_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94029 entries, B0012GTZCK to B01HJENY3Y
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         94029 non-null  object 
 1   brand         94029 non-null  object 
 2   genre         94029 non-null  object 
 3   print_length  94029 non-null  float64
 4   word_wise     94029 non-null  int64  
 5   lending       94029 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 5.0+ MB


In [13]:
df_meta_all.head()

,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B0012GTZCK,God in the White House,Randall Balmer,History,260.0,1,0
B0012LHGJ4,Vingt Mille Lieues sous les mers (French Editi...,Jules Verne,Reference,606.0,0,1
B0012RMVH0,Wood-Carving Design and Workmanship - Kindle e...,George Jack,Arts & Photography,310.0,0,0
B0012TAD1O,Souls Night (The Vampire Pacts) - Kindle edition,Kallysten,Romance,74.0,1,1
B0012U0NKE,The Misplaced Horse (1) eBook,Constance Downes,"Comics, Manga & Graphic Novels",444.0,0,1


In [14]:
model_df = dtm.merge(df_meta_all, left_index=True, right_index=True)
model_df.drop(columns=['title', 'brand', 'word_wise', 'lending'], inplace=True)
model_df = pd.get_dummies(model_df, columns=['genre'])
model_df.head()

,abl,absolut,action,actual,adventur,almost,along,alreadi,also,although,...,genre_Politics & Social Sciences,genre_Reference,genre_Religion & Spirituality,genre_Romance,genre_Science & Math,genre_Science Fiction & Fantasy,genre_Self-Help,genre_Sports & Outdoors,genre_Teen & Young Adult,genre_Travel
asin,,,,,,,,,,,,,,,,,,,,,
B0012GTZCK,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
B0012LHGJ4,0.0,0.0,0.000000,0.000000,0.362908,0.0,0.0,0.0,0.0,0.000000,...,0,1,0,0,0,0,0,0,0,0
B0012RMVH0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
B0012TAD1O,0.0,0.0,0.090261,0.090717,0.000000,0.0,0.0,0.0,0.0,0.098413,...,0,0,0,1,0,0,0,0,0,0
B0012U0NKE,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [15]:
cols = ['print_length']
scaler = MinMaxScaler()
scaled = scaler.fit_transform(model_df[cols])
scaled_column = pd.DataFrame(scaled, index=model_df.index, columns=cols)
model_df['print_length'] = scaled_column['print_length']

In [16]:
y = np.array(model_df.loc['B01HJBPUWA'])
y = y.reshape(1, -1)
cos_sim = cosine_similarity(model_df, y)
cos_sim = pd.DataFrame(data=cos_sim, index=model_df.index)
cos_sim.sort_values(by = 0, ascending=False, inplace=True)
results = cos_sim.head(11)

In [17]:
results

,0
asin,
B01HJBPUWA,1.000000
B00BG2WZZE,0.845608
B01DE64NKE,0.845022
B00C1MZN5C,0.843422
B00NGANDX2,0.839412
B00P5JNNBE,0.838702
B00B0071TC,0.838390
B005SFRJ6K,0.837444
B00B2QZH68,0.835785


In [18]:
def book_review_recommend():
    
    book = input('ASIN: ')
    n_recs = int(input('How many recommendations? '))
    
    y = np.array(model_df.loc[book]).reshape(1, -1)
    cos_sim = cosine_similarity(model_df, y)
    cos_sim = pd.DataFrame(data=cos_sim, index=model_df.index)
    cos_sim.sort_values(by = 0, ascending = False, inplace=True)
    results = cos_sim.head(n_recs+1).index.values[1:]
    results_df = df_meta_all.loc[results]
    return results_df

In [19]:
book_review_recommend()

ASIN: B01HJBPUWA
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B00BG2WZZE,The Sixth Science Fiction MEGAPACK&reg;,Johnston McCulley,Science Fiction & Fantasy,666.0,1,0
B01DE64NKE,The Doomsday Chronicles (Future Chronicles Bo...,Samuel Peralta,Science Fiction & Fantasy,438.0,1,0
B00C1MZN5C,The Grimm Diaries Prequels volume 11- 14,Cameron Jace,Science Fiction & Fantasy,159.0,1,1
B00NGANDX2,Donavan,DK Cassidy,Science Fiction & Fantasy,25.0,1,1
B00P5JNNBE,Some of the Best from Tor.com,Various Various Authors,Science Fiction & Fantasy,941.0,1,0
B00B0071TC,Omega Zero eBook,Ryan Henry,Science Fiction & Fantasy,228.0,1,1
B005SFRJ6K,Infinity Blade,Brandon Sanderson,Science Fiction & Fantasy,124.0,1,0
B00B2QZH68,The Runner (Silo Submerged Book 1) eBook,WJ Davies,Science Fiction & Fantasy,61.0,1,1
B00EK5IMOC,Kindreds,Tani Mura,Science Fiction & Fantasy,330.0,1,1


In [20]:
book_review_recommend()

ASIN: B00YN6XHMU
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B00I48KN16,BURN (Ignite Book 2) - Kindle edition,R.J. Lewis,Literature & Fiction,241.0,1,1
B00EV9LSJI,Tears of Tess (Monsters in the Dark Book 1) - ...,Pepper Winters,Literature & Fiction,380.0,1,1
B00GL6FUMU,Replace Me (Kin Series Book 2) - Kindle edition,Jennifer Foor,Literature & Fiction,358.0,1,0
B00ESJ3S94,Fueled (The Driven Series Book 2) - Kindle edi...,K. Bromberg,Literature & Fiction,353.0,1,1
B00EWRZG5Q,The Commitment (The Unrestrained Series Book 2...,S. E. Lund,Literature & Fiction,321.0,1,1
B00FA2B9S0,Who We Are (FireNine) - Kindle edition,S. Q. Williams,Literature & Fiction,317.0,1,1
B00JS7CSVA,A Test of Love (Interracial Erotic Romance BWW...,kenya wright,Literature & Fiction,282.0,1,1
B00LS610HC,Rock All Night (The Rock Star&#39;s Seduction ...,Olivia Thorne,Literature & Fiction,465.0,1,1
B00F0SI3E2,Off the Record - Kindle edition,K.A. Linde,Literature & Fiction,411.0,1,0


In [21]:
book_review_recommend()

ASIN: B00C2WDD5I
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B01AW25GRC,Doubt (Caroline Auden Book 1) - Kindle edition,C. E. Tobisman,Literature & Fiction,348.0,1,0
B0070CSKE4,"The Twelve Stones (The Twelve Stones, Book 1) ...",RJ Johnson,Literature & Fiction,354.0,1,1
B018SCGDWK,Dead Lawyers Don&#39;t Lie: A Gripping Thrille...,Mark Nolan,Literature & Fiction,602.0,1,1
B00KH3Z53W,Hacker For Hire (Ted Higuera Series Book 2) - ...,Pendelton Wallace,Literature & Fiction,484.0,1,1
B015GWEGGA,Golden Age (The Shifting Tides Book 1) eBook,James Maxwell,Literature & Fiction,466.0,1,0
B00Q0DGF7I,Adrift (Adrift Series Book 1) - Kindle edition,K.R. Griffiths,Literature & Fiction,384.0,1,1
B00NQLAPQE,On Distant Shores (Earth Exiles Book 1) eBook,Mark Harritt,Literature & Fiction,269.0,1,1
B006AXT6G0,Death in the Beginning (The God Tools Book 1) ...,Gary Williams,Literature & Fiction,333.0,1,1
B00DU2IYVM,Omega Dog (Joe Venn Crime Action Thriller Seri...,Tim Stevens,Literature & Fiction,283.0,1,1


In [22]:
book_review_recommend()

ASIN: B00IJYII4E
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B00MARJTYO,Slow Burn,Bobby Adair,"Mystery, Thriller & Suspense",607.0,1,1
B00EJR4HVS,Enemies (The Girl in the Box Book 7) - Kindle ...,Robert J. Crane,"Mystery, Thriller & Suspense",310.0,1,1
B00BCQ6PYQ,Omega (The Girl in the Box Book 5) - Kindle ed...,Robert J. Crane,"Mystery, Thriller & Suspense",245.0,1,1
B018JP6WE0,Love Lost (The Kurtherian Gambit Book 3) - Kin...,Michael Anderle,"Mystery, Thriller & Suspense",260.0,1,1
B00IGEIAB8,RANEY &amp; LEVINE (EMBRYO: A Raney &amp; Levi...,J.A. Schneider,"Mystery, Thriller & Suspense",228.0,1,1
B00D00M5NM,Broken (The Girl in the Box Book 6) - Kindle e...,Robert J. Crane,"Mystery, Thriller & Suspense",154.0,1,1
B00W0P387A,Dying for a Living Boxset: Books 1-3 of Dying ...,Kory M. Shrum,"Mystery, Thriller & Suspense",782.0,1,1
B00GXV16H6,The Joe Dillard Series Box Set (Books 1 throug...,Scott Pratt,"Mystery, Thriller & Suspense",1287.0,1,1
B00H3QQGV6,Ready to Kill (Nathan McBride Book 4) - Kindle...,Andrew Peterson,"Mystery, Thriller & Suspense",348.0,1,0


In [23]:
book_review_recommend()

ASIN: B015BIHKH6
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B01DPQ4736,The Viscount&#39;s Wallflower Bride: A Sweet &...,Lauren Royal,Romance,402.0,1,1
B0082CA0Z8,Sea Scoundrel (Knave of Hearts Book 1) - Kindl...,Annette Blair,Romance,308.0,1,1
B0051EZBRM,"Never a Mistress, No Longer a Maid (Kellington...",Maureen Driscoll,Romance,317.0,1,1
B014NGXDWS,Transcending Darkness - Kindle edition,Airicka Phoenix,Romance,600.0,1,0
B00586XP5S,All&#39;s Fair in Love &amp; Seduction (The El...,Beverley Kendall,Romance,138.0,1,1
B00HMXLW9Q,"Forever Betrothed, Never the Bride (Scandalous...",Christi Caldwell,Romance,327.0,1,1
B00CMY7028,Already His (The Caversham Chronicles Book 2) ...,Sandy Raven,Romance,468.0,1,1
B006I2RQT2,In Love&#39;s Territory: A Western Historical ...,Lucy Evanson,Romance,258.0,1,1
B00PUS8AEK,Bound,Cora Reilly,Romance,273.0,1,1


In [24]:
book_review_recommend()

ASIN: B00OBIB21I
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B005IBNIF0,Love for Sale - Kindle edition,Joe Brewster,Literature & Fiction,11.0,1,1
B006YGE4NS,The Old Man and the Sea - Kindle edition,Ernest Hemingway,Literature & Fiction,88.0,1,0
B00UL1QO1M,But It&#39;s Too Dangerous - Kindle edition,S. B. Sheeran,Literature & Fiction,75.0,1,1
B00KAMO6TO,THE MAN FROM U.N.C.L.E: The Spider&#39;s Web A...,Casandra Nightengale,Literature & Fiction,47.0,1,1
B005BCPGI8,Firebug - Kindle edition,Jon Hartling,Literature & Fiction,6.0,1,1
B008KP6H4A,Forget Me Not - Kindle edition,Wayne Thomas Batson,Literature & Fiction,27.0,1,1
B00IPCU9SS,"First There Wasn&#39;t, Then There Was - Kindl...",Troy Blackford,Literature & Fiction,106.0,1,1
B0046A9K5G,AUNTIE GETS TOUGHER (TV FICTION CLASSIC Book 7...,Sandy Thomas,Literature & Fiction,74.0,1,1
B013F0T8CC,Fertile Farmgirl Collection: 6 Fertile First T...,Candy Quinn,Literature & Fiction,126.0,1,1


In [25]:
book_review_recommend()

ASIN: B008WD24JM
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B00G4FPA3C,"EVERWUD Book Two - CASTLES, CLOWNS &amp; POTIO...",Marilyn Dieckmann,Children's eBooks,184.0,1,0
B0094KJK1C,The Adventures of Zeppi - A Penguin Story - #1...,Charlotte K. Omillin,Children's eBooks,46.0,1,1
B00KYC5TF0,The Eleventh Ring (Bartholomew the Adventurer...,Tom Hoffman,Children's eBooks,322.0,1,1
B010QVQ36M,Toby&#39;s Tails - Still Wagging (Fantasy Farm...,Susan Keefe,Children's eBooks,50.0,1,1
B00ER8ECL4,In the Magician&#39;s House (You Say Which Way...,DM Potter,Children's eBooks,134.0,1,1
B0065RFPS4,The Great Kieranski and the Bardbuy - Kindle e...,Cate Mara,Children's eBooks,110.0,1,0
B00LMKWME4,"Oliver and Jumpy - the Cat Series, Introductio...",Werner Stejskal,Children's eBooks,24.0,0,0
B004RUXQGE,The Society of Imaginary Friends: An epic YA f...,Kristen Pham,Children's eBooks,305.0,1,1
B00HNCTDJ2,Books for Kids: ABIGAIL AND THE JUNGLE ADVENTU...,Tali Carmi,Children's eBooks,36.0,1,1


In [ ]:
knn = NearestNeighbors(n_neighbors=10)
knn.fit(model_df)

In [40]:
def book_review_recommend_knn():
    
    book = input('ASIN: ')
    n_recs = int(input('How many recommendations? '))
    
    x = np.array(model_df.loc[book]).reshape(1, -1)
    results = knn.kneighbors(x, n_recs+1, return_distance=False).flatten()
    index = model_df.iloc[results].index.values[1:]
    results_df = df_meta_all.loc[index]
    return results_df

In [41]:
book_review_recommend_knn()

ASIN: B01HJBPUWA
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B00BG2WZZE,The Sixth Science Fiction MEGAPACK&reg;,Johnston McCulley,Science Fiction & Fantasy,666.0,1,0
B01DE64NKE,The Doomsday Chronicles (Future Chronicles Bo...,Samuel Peralta,Science Fiction & Fantasy,438.0,1,0
B00C1MZN5C,The Grimm Diaries Prequels volume 11- 14,Cameron Jace,Science Fiction & Fantasy,159.0,1,1
B00NGANDX2,Donavan,DK Cassidy,Science Fiction & Fantasy,25.0,1,1
B00P5JNNBE,Some of the Best from Tor.com,Various Various Authors,Science Fiction & Fantasy,941.0,1,0
B00B0071TC,Omega Zero eBook,Ryan Henry,Science Fiction & Fantasy,228.0,1,1
B005SFRJ6K,Infinity Blade,Brandon Sanderson,Science Fiction & Fantasy,124.0,1,0
B00B2QZH68,The Runner (Silo Submerged Book 1) eBook,WJ Davies,Science Fiction & Fantasy,61.0,1,1
B00EK5IMOC,Kindreds,Tani Mura,Science Fiction & Fantasy,330.0,1,1


In [42]:
book_review_recommend()

ASIN: B01HJBPUWA
How many recommendations? 10


,title,brand,genre,print_length,word_wise,lending
asin,,,,,,
B00BG2WZZE,The Sixth Science Fiction MEGAPACK&reg;,Johnston McCulley,Science Fiction & Fantasy,666.0,1,0
B01DE64NKE,The Doomsday Chronicles (Future Chronicles Bo...,Samuel Peralta,Science Fiction & Fantasy,438.0,1,0
B00C1MZN5C,The Grimm Diaries Prequels volume 11- 14,Cameron Jace,Science Fiction & Fantasy,159.0,1,1
B00NGANDX2,Donavan,DK Cassidy,Science Fiction & Fantasy,25.0,1,1
B00P5JNNBE,Some of the Best from Tor.com,Various Various Authors,Science Fiction & Fantasy,941.0,1,0
B00B0071TC,Omega Zero eBook,Ryan Henry,Science Fiction & Fantasy,228.0,1,1
B005SFRJ6K,Infinity Blade,Brandon Sanderson,Science Fiction & Fantasy,124.0,1,0
B00B2QZH68,The Runner (Silo Submerged Book 1) eBook,WJ Davies,Science Fiction & Fantasy,61.0,1,1
B00EK5IMOC,Kindreds,Tani Mura,Science Fiction & Fantasy,330.0,1,1
